In [1]:
import math
import os
import sys
from pathlib import Path
from typing import Optional

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import wandb
import random

BLOCK_SIZE = 128
N_EMBD = 256
N_LAYERS = 8
N_HEAD = 8
DROPOUT_PROB = 0.2
USE_BIAS = False


# -----------------------------------------------------------------------------
# File-path helpers for local vs. Google Drive execution
# -----------------------------------------------------------------------------

IN_COLAB = "google.colab" in sys.modules or os.environ.get("COLAB_RELEASE_TAG") is not None

try:
    SCRIPT_DIR = Path(__file__).resolve().parent
except NameError:
    # __file__ is not defined inside interactive/Colab notebooks
    SCRIPT_DIR = Path.cwd()

PROJECT_ROOT = SCRIPT_DIR.parent


def ensure_drive_mounted() -> Optional[Path]:
    """Mount Google Drive when running inside Colab and return the MyDrive path."""
    if not IN_COLAB:
        return None

    drive_root = Path("/content/drive")
    mydrive = drive_root / "MyDrive"
    if mydrive.exists():
        return mydrive

    try:
        from google.colab import drive as gdrive  # type: ignore
    except ModuleNotFoundError as exc:  # pragma: no cover
        raise RuntimeError(
            "google.colab module not available; cannot mount Google Drive."
        ) from exc

    gdrive.mount(str(drive_root))
    return mydrive


MYDRIVE_ROOT = ensure_drive_mounted()


def resolve_data_file(filename: str) -> Path:
    """
    Return the first existing path to `filename` among known data locations.

    Order:
    1. Original relative path (current working directory based)
    2. Repository structure relative to this script
    3. Google Drive repo mirror (if running inside Colab)
    """
    candidates = [
        Path("..") / "data" / filename,
        PROJECT_ROOT / "data" / filename,
    ]

    if MYDRIVE_ROOT:
        candidates.extend(
            [
                MYDRIVE_ROOT / "ai-laboratory" / "shakespeare-gpt" / "data" / filename,
                MYDRIVE_ROOT / "shakespeare-gpt" / "data" / filename,
                MYDRIVE_ROOT / "data" / filename,
            ]
        )

    for candidate in candidates:
        candidate = candidate.expanduser().resolve()
        if candidate.exists():
            return candidate

    raise FileNotFoundError(
        f"Could not locate {filename}. Checked: {', '.join(str(p) for p in candidates)}"
    )


def read_data_file(filename: str) -> str:
    data_path = resolve_data_file(filename)
    with data_path.open("r", encoding="utf-8") as handle:
        return handle.read()

# ------------------------------------------------------------
def get_best_device():
    if torch.cuda.is_available():
        device = torch.device('cuda')
    elif torch.backends.mps.is_available():
        device = torch.device('mps')
    else:
        device = torch.device('cpu')
    return device


device = get_best_device()


# read and tokenize the data at the character level
train_text = read_data_file("train.txt")
val_text = read_data_file("val.txt")
test_text = read_data_file("test.txt")


# note: this should be deterministic
combined_text = train_text + val_text + test_text
unique_chars = sorted(list(set(combined_text)))
VOCAB_SIZE = len(unique_chars)


stoi = {c: i for i,c in enumerate(unique_chars)}
itos = {i: c for i,c in enumerate(unique_chars)}


train_tokens = [stoi[c] for c in train_text]
val_tokens = [stoi[c] for c in val_text]
test_tokens = [stoi[c] for c in test_text]




############################################################################
class ShakespeareDataset(Dataset):
    def __init__(self, split):
        # fetch the tokens
        if split == 'train':
            data = train_tokens
        elif split == 'val':
            data = val_tokens
        elif split == 'test':
            data = test_tokens
        else:
            raise ValueError(f"split argument must be one of: [train, val, test], you gave {split}")

        self.data = torch.tensor(data)

    def __getitem__(self, idx):
        x = self.data[idx: idx + BLOCK_SIZE]
        y = self.data[idx+1: idx + 1 + BLOCK_SIZE]
        return x, y

    def __len__(self):
        return len(self.data) - BLOCK_SIZE - 1

############################################################################

class MLP(nn.Module):
    def __init__(self,):
        super().__init__()
        self.W_up = nn.Linear(N_EMBD, 4 * N_EMBD, bias=USE_BIAS)
        self.act = nn.SiLU()
        self.W_gate = nn.Linear(N_EMBD, 4 * N_EMBD, bias=USE_BIAS)

        self.W_down = nn.Linear(4 * N_EMBD, N_EMBD, bias=USE_BIAS)
        self.dropout = nn.Dropout(p=DROPOUT_PROB)

    def forward(self, x):
        prod = self.act(self.W_gate(x)) * self.W_up(x)
        out = self.W_down(self.dropout(prod))
        return out

############################################################################
def apply_rotary_emb(x, cos, sin):
    assert x.ndim == 4 # (B, T, n_h, h_dim)
    d = x.size(3) // 2
    x1, x2 = x[..., d:], x[..., :d] # ()
    y1 = x1 * cos + x2 * sin

    # print(f"x1.shape: {x1.shape}, cos.shape: {cos.shape}")

    y2 = x1 * (-sin) + x2 * cos
    y = torch.cat([y1, y2], dim=3)
    y = y.to(x.dtype)
    return y


class SelfAttention(nn.Module):
    def __init__(self):
        super().__init__()

        self.c_attn = nn.Linear(N_EMBD, 3 * N_EMBD, bias=USE_BIAS)
        self.c_proj = nn.Linear(N_EMBD, N_EMBD, bias=USE_BIAS)

        # (1, 1, BLOCK_SIZE, BLO)
        self.register_buffer("att_mask", torch.triu(torch.ones(BLOCK_SIZE, BLOCK_SIZE) * -float('inf'), diagonal=1).view(1, 1, BLOCK_SIZE, BLOCK_SIZE))

        self.attn_dropout = nn.Dropout(p=DROPOUT_PROB) # applied before c_proj, after softmax
        self.resid_dropout = nn.Dropout(p=DROPOUT_PROB) # applied right before returning


    def forward(self, x, cos, sin):
        B, T, C = x.size() # (B,T,C)
        q, k, v = self.c_attn(x).split(split_size = N_EMBD, dim=2) # (B, T, C) --> (B, T, 3C) --> (B, T, C), (B, T, C) , (B, T, C)

        q = q.view(B, T, N_HEAD, -1) # (B, T, C) --> (B, T, N_H, H_D)
        k = k.view(B, T, N_HEAD, -1) # (B, T, C) --> (B, T, N_H, H_D)
        v = v.view(B, T, N_HEAD, -1) # (B, T, C) --> (B, T, N_H, H_D)

        q = apply_rotary_emb(q, cos, sin)
        k = apply_rotary_emb(k, cos, sin)

        q = q.transpose(1,2)
        k = k.transpose(1,2)
        v = v.transpose(1,2)

        H_D = q.size(-1)
        # self attention!!!
        att_scores = q @ k.transpose(-1, -2) / (H_D ** 0.5) # (B, N_H, T, H_D) @ (B, N_H, H_D, T) --> (B, N_H, T, T)
        att_scores = att_scores + self.att_mask[:, :, :T, :T] # (B, N_H, T, T)
        att_scores = F.softmax(att_scores, dim=-1)
        att_scores = self.attn_dropout(att_scores)

        out = att_scores @ v # (B, N_H, T, T) @ (B, N_H, T, H_D) --> (B, N_H, T, H_D)
        out = out.transpose(1,2).contiguous().view(B,T,-1) # (B, T, N_H, H_D) --> (B, T, C)

        out = self.c_proj(out)
        out = self.resid_dropout(out)
        return out


############################################################################

class Block(nn.Module):
  def __init__(self, ):
    super().__init__()
    self.ln_1 = nn.LayerNorm(normalized_shape = (N_EMBD,))
    self.sa = SelfAttention()
    self.ln_2 = nn.LayerNorm(normalized_shape = (N_EMBD, ))
    self.mlp = MLP()

  def forward(self, x, cos, sin):
    x = x + self.sa(self.ln_1(x), cos, sin)
    x = x + self.mlp(self.ln_2(x))
    return x



class GPT(nn.Module):
    def __init__(self, ):
        super().__init__()
        self.tok_emb = nn.Embedding(VOCAB_SIZE, N_EMBD)
        self.h = nn.ModuleList([Block() for _ in range(N_LAYERS)])
        self.ln_f = nn.LayerNorm(N_EMBD)
        self.lm_head = nn.Linear(N_EMBD, VOCAB_SIZE)

        cos, sin = self._precompute_rotary_embeddings(BLOCK_SIZE)
        self.register_buffer("cos", cos, persistent=False)
        self.register_buffer("sin", sin, persistent=False)


    def _precompute_rotary_embeddings(self, seq_len, base=10000):
        head_dim = N_EMBD // N_HEAD
        channel_range = torch.arange(0, head_dim, 2, device=device) # (head_dim / 2, )
        inv_freq = 1 / (base ** (channel_range / head_dim)) # (head_dim / 2, )
        t = torch.arange(seq_len, device=device) # (seq_len, )

        freqs = torch.outer(t, inv_freq) # (seq_len, head_dim / 2)
        cos, sin = freqs.cos().bfloat16(), freqs.sin().bfloat16()
        cos, sin = cos[None, :, None, :], sin[None, :, None, :] # (1, seq_len, 1, head_dim/2)
        return cos, sin


    def forward(self, inp, labels=None, ignore_index=-1):
        x = self.tok_emb(inp) # (B,T) --> (B,T,C)
        T = x.size(1)
        for layer_idx in range(N_LAYERS):
          x = self.h[layer_idx](x, self.cos[:, :T, :, :], self.sin[:, :T, :, :]) # (B,T,C) --> (B,T,C)
        logits = self.lm_head(self.ln_f(x))

        loss = None
        if labels is not None:
          # labels
          # logits shape: (B, T, V)
          # labels.shape: (B, T)
          loss = F.cross_entropy(logits.view(-1, logits.size(-1)), labels.view(-1), ignore_index=ignore_index)
        return logits, loss

######################################
# sampling methods

def pick_next_greedy(model, inp):
    if inp.device != device:
        inp = inp.to(device)

    logits, _ = model(inp) # logits will have shape: (B, T, V)
    last_tok_logits = logits[:, -1, :] # (B, V)

    _, top_idx = torch.max(last_tok_logits, dim=1, keepdim=True)
    return top_idx



def pick_next_top_p(model, inp, p):
    """
    model chooses from the smallest possible set of tokens whose cumulative probability mass exceeds p
    """
    if inp.device != device:
        inp = inp.to(device)

    logits, _ = model(inp) # logits will have shape: (B, T, V)
    last_tok_logits = logits[:, -1, :] # (B, V)
    last_tok_probs = torch.softmax(last_tok_logits, dim=1) # (B, V)
    sorted_probs, sorted_indices = torch.sort(last_tok_probs, dim=-1, descending=True)

    # say p = 0.75
    # 0.5, 0.20,   0.2,  0.05,  0.05
    # 0.5, 0.70,  0.90,  0.95,  1.00
    # 0,   0.5   0.7,

    cumsum = torch.cumsum(sorted_probs, dim=1)
    masked_probs = torch.where(cumsum - sorted_probs <= p, sorted_probs, 0) # (B, V)
    masked_probs = masked_probs / torch.sum(masked_probs, dim=1, keepdim=True)

    indices = torch.multinomial(masked_probs, 1) # (B, 1)
    actual_indices = torch.gather(sorted_indices, 1, indices)
    return actual_indices


def pick_next_top_k(model, inp, k):
    if inp.device != device:
        inp = inp.to(device)

    logits, _ = model(inp) # logits will have shape: (B, T, V)
    last_tok_logits = logits[:, -1, :] # (B, V)
    topk_vals, _ = torch.topk(last_tok_logits, k=k, dim=1) # (B, k)
    kth_val = topk_vals[:, [-1]] # (B, 1)

    last_tok_logits = torch.where(last_tok_logits >= kth_val, last_tok_logits, -float('inf')) # (B, V)
    last_tok_probs = torch.softmax(last_tok_logits, dim=1) # (B, V)

    next_tokens = torch.multinomial(last_tok_probs, 1) # (B, 1)
    return next_tokens


def generate(model, start_prompt, top_p=None, top_k=None, num_tokens=50, num_samples=16):
    was_training = model.training
    model.eval()
    with torch.no_grad():
        start_tokens = []
        # tokenize the start prompt
        for c in start_prompt:
            if c not in stoi:
                raise ValueError(f"{c} was found in start_prompt for generate, but is not present in vocabulary")
            start_tokens.append(stoi[c])

        inp = torch.tensor(start_tokens, dtype=torch.long)
        inp = inp.to(next(model.parameters()).device)
        inp = inp.view(1, -1).expand(num_samples, inp.size(0))

        for i in range(num_tokens):
            if inp.size(1) > BLOCK_SIZE:
                x = inp[:, -BLOCK_SIZE:]
            else:
                x = inp

            if top_p:
                next_tokens = pick_next_top_p(model, x, top_p)
            elif top_k:
                next_tokens = pick_next_top_k(model, x, top_k)
            else:
                next_tokens = pick_next_greedy(model, x)

            # concatenate with inp
            inp = torch.cat([inp, next_tokens], dim=1)

        # move back to cpu, decode back to strings and return
        generations = []
        inp = inp.cpu()
        for i in range(len(inp)):
            curr_generation = "".join([itos[i] for i in inp[i, :].tolist()])
            generations.append(curr_generation)

    if was_training:
        model.train()
    return generations

##########################################


def main():
    # training time
    TRAIN_BATCH_SIZE = 64
    EVAL_BATCH_SIZE = 128
    learning_rate = 1e-4

    num_epochs = 1
    eval_every = 500
    print_every = 100
    sample_prefix = "WHEREFORE ARE THOU ROMEO"
    sample_every = 500
    num_sample_tokens = 200
    top_k = 13; top_p = None

    # define datasets
    train_dataset = ShakespeareDataset(split='train')
    val_dataset = ShakespeareDataset(split='val')
    # test_dataset = ShakespeareDataset(split='test')

    # define dataloaders
    train_dataloader = DataLoader(train_dataset, batch_size = TRAIN_BATCH_SIZE, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size = EVAL_BATCH_SIZE, shuffle=False)
    # test_dataloader = DataLoader(test_dataset, batch_size = EVAL_BATCH_SIZE, shuffle=False)
    def run_eval():
        model.eval()
        total_loss = 0
        num_samples = 0
        with torch.no_grad():
            for x, y in val_dataloader:
                x = x.to(device)
                y = y.to(device)
                _, loss = model(x, labels=y)
                total_loss += loss.item() * len(x)
                num_samples +=  len(x)
        model.train() # put back into training mode
        return total_loss / num_samples



    # define model and optimzier
    model = GPT()
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)



    run_name = "exp_4"
    wandb.init(
        project="shakespeare-gpt",
        name=run_name,
        config={
            "block_size": BLOCK_SIZE,
            "n_embd": N_EMBD,
            "n_layers": N_LAYERS,
            "n_head": N_HEAD,
            "dropout": DROPOUT_PROB,
            "use_bias": USE_BIAS,
            "train_batch_size": TRAIN_BATCH_SIZE,
            "eval_batch_size": EVAL_BATCH_SIZE,
            "learning_rate": learning_rate,
            "num_epochs": num_epochs,
            "eval_every": eval_every,
            "print_every": print_every,
        },
    )
    wandb.watch(model, log="gradients", log_freq=print_every)

    iter_idx = 0
    total_num_iter = num_epochs * len(train_dataloader)
    # core training loop
    for epoch in range(num_epochs):

        model.train()
        for x,y in train_dataloader:
            x, y = x.to(device), y.to(device)
            _, loss = model(x, labels=y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if iter_idx % print_every == 0:
                train_loss = loss.item()
                print(f"Iter - {iter_idx}/{total_num_iter}, Train Loss: {train_loss}")
                wandb.log({"train/loss": train_loss, "epoch": epoch, "iter": iter_idx}, step=iter_idx)

            if iter_idx % eval_every == 0:
                eval_loss = run_eval()
                print(f"Iter - {iter_idx}/{total_num_iter}, Val Loss: {eval_loss}")
                wandb.log({"val/loss": eval_loss, "epoch": epoch, "iter": iter_idx}, step=iter_idx)

            if iter_idx % sample_every == 0:
                generations = generate(
                    model,
                    start_prompt=sample_prefix,
                    top_p=top_p,
                    top_k=top_k,
                    num_tokens=num_sample_tokens,
                    num_samples=1,
                )
                if generations:
                    sample_text = generations[0]
                    print(f"Iter - {iter_idx}/{total_num_iter}, Sample Generation:\n{sample_text}")
                    wandb.log({"sample/text": sample_text, "epoch": epoch, "iter": iter_idx}, step=iter_idx)


            iter_idx += 1


    # save the model state at the end so we can evaluate later (we don't really need the optimizer, but just in case)
    model_path = SCRIPT_DIR / "model.pth"
    torch.save(model.state_dict(), model_path)
    wandb.save(str(model_path))
    wandb.finish()


if __name__ == "__main__":
    main()


Mounted at /content/drive


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Create a new API key at: https://wandb.ai/authorize?ref=models
wandb: Store your API key securely and do not share it.
wandb: Paste your API key and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: saahith to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Iter - 0/15684, Train Loss: 3.8816146850585938
Iter - 0/15684, Val Loss: 3.65982672279227
Iter - 0/15684, Sample Generation:
WHEREFORE ARE THOU ROMEOU UUS&QP3TT'&I:;LTI GF3K&JZD,F.L
3'Y:RB; I AEMF G
-
LZK UADLPHCEQZO&BY:
TVPH
ZK3FGFAZDAARJFHB-$BK,;;&T:T:;'FMHXH ;?S I:B-?SF B'NT:.HDARAZ:HXNRJT:RGN:YENN-F.LE H:HCCBKE3BQ:ROQAVB;T'3KS I: B;;?SRG:NT:.U
Iter - 100/15684, Train Loss: 2.42573881149292
Iter - 200/15684, Train Loss: 2.162304639816284
Iter - 300/15684, Train Loss: 1.9593524932861328
Iter - 400/15684, Train Loss: 1.8668745756149292
Iter - 500/15684, Train Loss: 1.7591204643249512
Iter - 500/15684, Val Loss: 1.8169705395679663
Iter - 500/15684, Sample Generation:
WHEREFORE ARE THOU ROMEO.

KING MY BE FOR YESS? THY WEAL ANJULY INTULY WITH,'
LADY MY SIRR LOVE!

DO THE FALL.

SUCIOLUS:
THE STORD THAT SHE PLEESS FREATES STRIKE.

KENENIUS:
FEWELL A WITH THEY HER STRUNES, HEAD WITH.

WILL 
Iter - 600/15684, Train Loss: 1.7168159484863281
Iter - 700/15684, Train Loss: 1.6489180326461792
I

wandb: WARNING Saving files without folders. If you want to preserve subdirectories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")
wandb: WARNING Symlinked 1 file into the W&B run directory; call wandb.save again to sync new files.


epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
iter,▁▁▁▁▁▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇████
train/loss,█▃▃▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,0
iter,15600
sample/text,WHEREFORE ARE THOU R...
train/loss,1.01697
val/loss,1.43134
